In [1]:
# -*- coding: utf-8 -*-
"""
Created on Mon Apr 14 09:33:08 2025

@author: taupiacs
"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import dm4bem

# Data
# ====
# dimensions
L, l, H, w = 7, 5, 4, 0.20  # m

# thermo-physical propertites
λ = 1.7             # W/(m K) wall thermal conductivity
# λwood = 0.12 # W/(m K) chestnut thermal conductivity
λglass = 2 # W/(m K) glass thermal conductivity
ρ = 1.2 # kg/m3 density
c = 1000   #  J/(kg K) specific heat air
hi, ho = 8, 25      # W/(m2 K) convection coefficients in, out

# outdoor/indoor temperature
To = 40             # °C
To_excel = To      


# ventilation rate (air-changes per hour)
ACH = 2             # volume/h

V_dot = L * l * H * ACH / 3600  # volumetric air flow rate
m_dot = ρ * V_dot               # mass air flow rate


legntwindow = l/2  # m length of the window
windowheight = H / 2
Sg = legntwindow* windowheight # m² surface area of the glass 

Sw = 3*(L*H) + 2*(l*H) - Sg # m² surface area of the wall

Swood = L/5 * (3/4)*H

air = {'Density': 1.2,                      # kg/m³
       'Specific heat': 1000}               # J/(kg·K)
pd.DataFrame(air, index=['Air'])

concrete = {'Conductivity': 1.400,          # W/(m·K)
            'Density': 2300.0,              # kg/m³
            'Specific heat': 880,           # J/(kg⋅K)
            'Width': 0.2,                   # m
            'Surface': Sw}            # m²

insulation = {'Conductivity': 0.04,        # W/(m·K)
              'Density': 55.0,              # kg/m³
              'Specific heat': 1210,        # J/(kg⋅K)
              'Width': 0.08,                # m
              'Surface': Sw}          # m²

glass = {'Conductivity': 1.05,               # W/(m·K)
         'Density': 2500,                   # kg/m³
         'Specific heat': 1210,             # J/(kg⋅K)
         'Width': 0.04,                     # m
         'Surface': Sg}                   # m²

wood = {'Conductivity': 0.12,               # W/(m·K)
         'Density': 560,                   # kg/m³
         'Specific heat': 1210,             # J/(kg⋅K)
         'Width': 0.04,                     # m
         'Surface': Swood}                   # m²

wall = pd.DataFrame.from_dict({'Layer_out': concrete,
                               'Layer_in': insulation,
                               'Glass': glass,
                               'Wood': wood},
                              orient='index')
wall

# radiative properties
ε_wLW = 0.85    # long wave emmisivity: wall surface (concrete)
ε_gLW = 0.90    # long wave emmisivity: glass pyrex
α_wSW = 0.25    # short wave absortivity: white smooth surface
α_gSW = 0.38    # short wave absortivity: reflective blue glass
τ_gSW = 0.30    # short wave transmitance: reflective blue glass

# Solar radiation

# in summer
# Gsolar = 1000       # W/m²
# in winter
Gsolar = 300        # W/m²

E = Gsolar*α_wSW    # W/m2 wall
Eglass = Gsolar*α_gSW  # W/m2 window

Φo = E*Sw                     # outside wall heating
Φi = Gsolar*τ_gSW*α_wSW * Sw  # inside wall heating though glass
Φa = Eglass * Sg              # solar radiation absorbed by the glass 
Qa = 0                        # no other heating (only solar)


σ = 5.67e-8     # W/(m²⋅K⁴) Stefan-Bolzmann constant

h = pd.DataFrame([{'in': 8., 'out': 25}], index=['h'])  # W/(m²⋅K)
h

# conduction
G_cd = wall['Conductivity'] / wall['Width'] * wall['Surface']
pd.DataFrame(G_cd, columns=['Conductance'])

# convection
Gw = h * wall['Surface'].iloc[0]     # wall
Gg = h * wall['Surface'].iloc[2]     # glass

# view factor wall-glass
Fwg = glass['Surface'] / concrete['Surface']


# T_int = 273.15 + np.array([0, 40])
# coeff = np.round((4 * σ * T_int**3), 1)
# print(f'For 0°C < (T/K - 273.15)°C < 40°C, 4σT³/[W/(m²·K)] ∈ {coeff}')

# T_int = 273.15 + np.array([10, 30])
# coeff = np.round((4 * σ * T_int**3), 1)
# print(f'For 10°C < (T/K - 273.15)°C < 30°C, 4σT³/[W/(m²·K)] ∈ {coeff}')

# T_int = 273.15 + 20
# coeff = np.round((4 * σ * T_int**3), 1)
# print(f'For (T/K - 273.15)°C = 20°C, 4σT³ = {4 * σ * T_int**3:.1f} W/(m²·K)')

# =============================================================================
# long wave radiation not considered
# =============================================================================
# # long wave radiation
# Tm = 20 + 273   # K, mean temp for radiative exchange

# GLW1 = 4 * σ * Tm**3 * ε_wLW / (1 - ε_wLW) * wall['Surface']['Layer_in']
# GLW12 = 4 * σ * Tm**3 * Fwg * wall['Surface']['Layer_in']
# GLW2 = 4 * σ * Tm**3 * ε_gLW / (1 - ε_gLW) * wall['Surface']['Glass']

# GLW = 1 / (1 / GLW1 + 1 / GLW12 + 1 / GLW2)

# =============================================================================
# modified
# =============================================================================
# ventilation flow rate
Va = L*l*H                   # m³, volume of air
ACH = 2                     # 1/h, air changes per hour
Va_dot = ACH / 3600 * Va    # m³/s, air infiltration

# ventilation & advection
Gv = air['Density'] * air['Specific heat'] * Va_dot

# =============================================================================
# no controller considered in the thermal circuit
# =============================================================================
# P-controler gain
# Kp = 1e4            # almost perfect controller Kp -> ∞
# Kp = 1e-3           # no controller Kp -> 0

# glass: convection outdoor & conduction
Ggs = float(1 / (1 / Gg.loc['h', 'out'] + 1 / (2 * G_cd['Glass'])))

C = wall['Density'] * wall['Specific heat'] * wall['Surface'] * wall['Width']
pd.DataFrame(C, columns=['Capacity'])

C['Air'] = air['Density'] * air['Specific heat'] * Va
pd.DataFrame(C, columns=['Capacity'])

# =============================================================================
# our thermal circuit
# =============================================================================

# temperature nodes
θ = ['θ0', 'θ1', 'θ2', 'θ3', 'θ4', 'θ5', 'θ6', 'θ7']

# flow-rate branches
q = ['q0', 'q1', 'q2', 'q3', 'q4', 'q5', 'q6', 'q7', 'q8', 'q9', 'q10']

# temperature nodes
nθ = 8      # number of temperature nodes
θ = [f'θ{i}' for i in range(nθ)]

# flow-rate branches
nq = 11     # number of flow branches
q = [f'q{i}' for i in range(nq)]

A = np.zeros([nq, nθ])      # n° of branches X n° of nodes
A[0, 0] = 1                 # branch 0: -> node 0
A[1, 0], A[1, 1] = -1, 1    # branch 1: node 0 -> node 1
A[2, 1], A[2, 2] = -1, 1    # branch 2: node 1 -> node 2
A[3, 2], A[3, 3] = -1, 1    # branch 3: node 2 -> node 3
A[4, 3], A[4, 4] = -1, 1    # branch 4: node 3 -> node 4
A[5, 4], A[5, 5] = -1, 1    # branch 5: node 4 -> node 5
A[6, 5] = 1
A[7, 6] = 1
A[8, 6], A[8, 7] = -1, 1
A[9, 5], A[9, 7] = 1, -1
A[10, 5] = 1

pd.DataFrame(A, index=q, columns=θ)

U_wood = 2 # W/(m².K)
G_wood = U_wood * Swood
 
G = np.array(np.hstack(
    [Gw['out'],
     2 * G_cd['Layer_out'], 2 * G_cd['Layer_out'],
     2 * G_cd['Layer_in'], 2 * G_cd['Layer_in'],
     Gw['in'],
     G_wood,
     Ggs,
     2 * G_cd['Glass'],
     Gg['in'],
     Gv]))
# add Kp then

# np.set_printoptions(precision=3, threshold=16, suppress=True)
# pd.set_option("display.precision", 1)
pd.DataFrame(G, index=q)

neglect_air_glass = False

if neglect_air_glass:
    C = np.array([0, C['Layer_out'], 0, C['Layer_in'], 0, 0,
                  0, 0])
else:
    C = np.array([0, C['Layer_out'], 0, C['Layer_in'], 0,
                 C['Air'], C['Glass'], 0])

# pd.set_option("display.precision", 3)
pd.DataFrame(C, index=θ)

b = pd.Series(['To', 0, 0, 0, 0, 0, 'To', 'To', 0, 0, 'To'],
               index=q)

f = pd.Series(['Φo', 0, 0, 0, 'Φi', 'Qa', 'Φa', 0],
              index=θ)

y = np.zeros(nθ)         # nodes
y[[5]] = 1              # nodes (temperatures) of interest
pd.DataFrame(y, index=θ)

# thermal circuit
A = pd.DataFrame(A, index=q, columns=θ)
G = pd.Series(G, index=q)
C = pd.Series(C, index=θ)
b = pd.Series(b, index=q)
f = pd.Series(f, index=θ)
y = pd.Series(y, index=θ)

TC = {"A": A,
      "G": G,
      "C": C,
      "b": b,
      "f": f,
      "y": y}

TC_steady = dm4bem.file2TC('./toy_model/TC_steady.csv', name='', auto_number=False)

# TC['G']['q11'] = 1e3  # Kp -> ∞, almost perfect controller
# TC['G']['q11'] = 0      # Kp -> 0, no controller (free-floating)

[As, Bs, Cs, Ds, us] = dm4bem.tc2ss(TC)
us



q0     To
q6     To
q7     To
q10    To
θ0     Φo
θ4     Φi
θ5     Qa
θ6     Φa
dtype: object